<a href="https://colab.research.google.com/github/salarbalou/steadystate_analysis/blob/main/Itr_vs_CQD_Comcemtration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import requests as req
import plotly.io as pio
import plotly.graph_objects as go
import re
import sympy as sy
from sympy import symbols, diff
from sympy.solvers import solve
from scipy.misc import derivative
import string as sr
pio.templates.default ='plotly_dark'
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
def findmb(x, y): # a simple function to find the slope and intecept of a line
  m,b = np.polyfit(x, y, deg= 1)
  return m,b

In [ ]:

CQD_C= ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'] # a list of CQD Concentartions

all_eta = [] # aan empty list of eta values (photothermal conversion efficiency) that can bes used to store the values of LPS generated in the for loop

for C in CQD_C: # since we have multiple dutycylce values we make a for loop
 baseurl = 'https://raw.githubusercontent.com/salarbalou/steadystate_analysis/main/CQD' # base url of the text file 
 char = 'X-t-T-dc-Itr.txt' # format of the file
 text = req.get(baseurl+C+char).text # we download and store the text of each file.txt
 data_path = re.compile(r'^([0-9.]+)[\s,]+([0-9.]+)+[\s,]+([-0-9.]+[\s,]+([-0-9.]+[\s,]+([-0-9.]+)', re.MULTILINE) # we grab the desired text within the text file (all data except for space and comma)
 data_list = data_path.findall(text) # this is a list of quadruple strings: each quadruple in this list have abcd string in it. 
 data = []
 for pentapole in data_list:
      eachpentapole = [] # this list here stores the float version of abcd in each quadruple: eachquadruple is a list of four float values
      for abcde in pentapole:
           eachpentapole.append(float(abcde))
      data.append((eachpentapole[0], eachpentapole[1], eachpentapole[2], eachpentapole[3], eachpentapole[4])) # we append all eachquadruples in data
 final_data = np.array(data) # here we make an array of the list: final_data is a matrix with 183 rows and 4 columns (t, T, I, dc)
 time = final_data[:, 0]
 temperature = final_data[:, 1]
 dutycycle = final_data[:, 2]
 Itr = final_data[:, 3]
 I0 = final_data[:, 4]
 beam_height = 0.7 #cm
 beam_width = 0.5 #cm



 #I0 = #incident laser power J/s /////Can be increased by increasing laser power
 #Itr = #transmitted laser power J/s ////can be decreased by increasing the concentration AND by overlapping adsorption wl on nanoparticle and lser wl 
 #eta = # photothermal conversion eficiency #coefficient # no units //// is a fucntion of nnaopartcle type
 #h = #heat transfor coefficient between solution and environment+ J/m^2*k*s
 #S = #surface area seperating solution from environemnt +m^2
 #Tw = #solution temp + K
 #T0 = #environment temp + K
 #m = #mass of solition + gr
 #Cp = #specific heat capacity of solution + J/g.K
 #v= volume of the CQD used in microliter + m^3
 #R = heating rate +K/s
 r = symbols('r')
 I0, Itr, eta, h, S, Tw, T0, m, Cp, R = symbols('I0, Itr, eta, h, S, Tw, T0, m, Cp, R')

 T0 = np.min(temperature)+273.15 #K
 Tw = np.max(temperature)+273.15 #K
 rho = 1 #density of water g/cm^3
 V= #volume of a part of solution thorugh which the laser beam is passing
 m= rho*1e-2 #gr based on density of water to be 1g/cm^3
 R = 1.5*1e-3 #radius of the volume filled with CQDs for 10ul CQD #m
 H  = 3.2*1e-3 # height of the volume filled with CQDs for 10ul CQD #m
 hmin = 598*1e-3 # W/mK at 20C
 hmax = 677*1e-3 # W/mK at 100
 mh, bh = findmb(np.arange(20, 100, 1)+273, np.arange(hmin, hmax, 1e-3))
 h = np.max((mh*(temperature[22:112]+273.15)+bh))  #W/m^2*k
 S = np.pi*R*(R+np.sqrt((H)**2+(R)**2)) #Surface Area of cone filled with CQD #m^2
 rate = findmb(np.arange(np.sort(temperature[22:112]).size), temperature[22:112]+273.15)[0] #K/s
 Cp = 4.184 #J/g.K for water
 eta = (rate*(Cp*m)+np.sum((h*S*(Tw - T0))))/(I0-Itr) # Equation to find the photothermal conversion efficiency #unit less # coefficient
 all_eta.append(eta)




In [ ]:
m,b=np.polyfit(np.arange(20, 100, 10), np.array(all_equ_temp), deg=1) # this function 
def givemedc(T):
  req_dc = (T-b)/m
  return np.int(req_dc)
givemedc(65) 
# Please note that this function only return the required DutyCylce for equ T for CQD of 10x concentration,
# in case of increase in concentration this value must increase (10 points for halfed-experimentally calculated)

45

In [ ]:
 fig = make_subplots(rows = 1, cols = 2, subplot_titles=('Temperature vs time for CQD'+C+'X')
 fig.add_scatter(x = time, y = temperature, mode = 'markers+lines', marker_color = 'red', row = 1, col = 1, name = 'Temperature vs time')
 fig.add_scatter(x = time, y = diffTt, mode = 'markers+lines', marker_color ='red' , line_color = "green", row  = 1, col  =2, name='dT/dt vs time')
 fig.update_xaxes(title_text = 'time (seconds)', row = 1, col = 1)
 fig.update_xaxes(title_text = 'time (seconds)', row = 1, col = 2)
 fig.update_yaxes(title_text = 'Temperature', row = 1, col = 1)
 fig.update_yaxes(title_text = 'dT/dt', row = 1, col = 2)
 fig.update_layout(height = 800, width = 2400)
 fig.show()

In [ ]:
if (np.array(all_equ_temp).size>1): # this will show the Temperature vs DutyCycle plot only in case if we have more than 1 utilized dutycycle
 fig2 = make_subplots(specs=[[{"secondary_y": True}]])
 fig2.add_trace(go.Scatter(x = all_laser_power_density, y= all_equ_temp, mode = 'markers+lines',name = 'Temperature vs Laser Power Density', marker_color = 'red'), secondary_y = False)
 fig2.add_trace(go.Scatter(x = all_laser_power_density, y= all_eta, mode = 'markers+lines', name = 'Photothermal Converson Effeciency vs Laser Power Density', marker_color = 'blue'), secondary_y = True)
 fig2.update_layout(height = 800, width= 2400)
 fig2.update_xaxes(title = "Laser Power Density (W/cm^2)")
 fig2.update_yaxes(title = 'Temperature', secondary_y=False)
 fig2.update_yaxes(title = 'Photothermal Conversion Efficiency', secondary_y=True)
 fig2.show()
else:
 print('No Results')

## Photothermal Theory:
Plasma is hot matter, in which electron are ripped apart from the 

*   When energy is given to plasma, it starts to oscilliate 
*   this phenomena is called plasmon oscilliation which froms a quasi particle called plasmon


*   in other words, plasmon is the quantum of plasma oscilliaiton 
*   similliar to photons, the quantum particles of optical oscilliaiotn


*   plasmonic resonance is mostly seen in electrons due to the abundacne of free free electron carriers (metals are conductive!)
*   However, plasmonic resonacne can also be seen in semiconductor of nanocrystals (ex,CQDs) that have hih concnetration of free electon carriers. 


*   quantumm dots are artificial atoms, small semiconductor particles
*   when photon couples with electron, electron excites from its valence band to condcution band, which makes a hole in the former.
since the size of the quantum dot is much smaller than the excitation wevelenght, the hole and excited electron can couple and froma quasi particle called exciton. This quasiparticel has high energy compared to the particle which is larger. 








